In [1]:
pip install pynput opencv-python numpy pytesseract mss


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install gymnasium


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pyautogui

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install 'stable-baselines3[extra]'


Note: you may need to restart the kernel to use updated packages.


In [5]:
import cv2
import numpy as np
import pytesseract
from pynput.mouse import Button, Controller as MouseController
from pynput.keyboard import Key, Controller as KeyboardController
from mss import mss
import time
import random

# intialize mouse and kryboard
keyboard = KeyboardController()
mouse = MouseController()

# pytesseract route
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'  

def press_y():
  
    keyboard.press('y')
    time.sleep(0.1)
    keyboard.release('y')

def random_move_and_click():

    x = random.randint(100, 800)  
    y = random.randint(100, 600)
    mouse.position = (x, y)
    mouse.click(Button.right)

def capture_screen():
  
    with mss() as sct:
        monitor = {"top": 0, "left": 1600, "width": 320, "height": 180} 
        screenshot = sct.grab(monitor)
        
        return np.array(screenshot)[:, :, :3]


def extract_numbers(image):
    # extract numbers from image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    text = pytesseract.image_to_string(thresh, config='outputbase digits')
    return text

def main():

    press_y()  
    while True:
        random_move_and_click()  
        screenshot = capture_screen()
        cv2.imshow('Game Capture', screenshot)  
        if cv2.waitKey(1) & 0xFF == ord('q'):  
            break
        numbers = extract_numbers(screenshot)
        print(f"Detected Numbers: {numbers}")
        time.sleep(2)  
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()


Detected Numbers: 
Detected Numbers: 
Detected Numbers: 
Detected Numbers: 
Detected Numbers: 
Detected Numbers: 



KeyboardInterrupt



In [ ]:
import numpy as np
from PIL import Image
import time
import pytesseract
import cv2
from mss import mss

time.sleep(5) 

def capture_specific_area(top, left, width, height):

    with mss() as sct:
        monitor = {"top": top, "left": left, "width": width, "height": height}
        screenshot = sct.grab(monitor)
        return np.array(screenshot)[:, :, :3]

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return thresh


def extract_numbers(image, psm=6):
    custom_config = f'--oem 3 --psm {psm} digits'
    text = pytesseract.image_to_string(image, config=custom_config)
    return text

def main():
    
    top1, left1, width1, height1 = 7, 2567, 19, 33  
    top2, left2, width2, height2 = 7, 2597, 19, 33
    top3, left3, width3, height3 =1865, 1400, 80, 40

    kills = capture_specific_area(top1, left1, width1, height1)
    deaths = capture_specific_area(top2, left2, width2, height2)
    minions = capture_specific_area(top3, left3, width3, height3)
    
    
    kills_preprocessed = preprocess_image(kills)
    deaths_preprocessed = preprocess_image(deaths)
    minions_preprocessed = preprocess_image(minions)
    
    # transform numpy to PIL
    #kills_image = Image.fromarray(kills_preprocessed)
    #deaths_image = Image.fromarray(deaths_preprocessed)
    minions_image = Image.fromarray(minions_preprocessed)
    
    # show image
    #kills_image.show()
    #deaths_image.show()
    minions_image.show()
    
    # can adjust PSM
    print("Kills:", extract_numbers(kills_preprocessed, 7))
    print("Deaths:", extract_numbers(deaths_preprocessed, 8))
    print("Minions:", extract_numbers(minions_preprocessed, 6))

if __name__ == "__main__":
    main()

In [5]:
pip install gymnasium --upgrade


Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import cv2
import pyautogui
import time
import random
from mss import mss
import pytesseract
from PIL import Image
import gymnasium as gym
from gymnasium import spaces
from pynput.mouse import Button, Controller as MouseController
from pynput.keyboard import Key, Controller as KeyboardController
import io






class MOBAEnv(gym.Env):
    def __init__(self):
        super(MOBAEnv, self).__init__()
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(low=0, high=255, shape=(1964, 3024, 3), dtype=np.uint8)
        self.last_action = None
        
        self.mouse = MouseController()
        self.keyboard = KeyboardController()
        pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract' 
        self.icons_paths = [
            '/Users/tim/Desktop/one more game.png',
            '/Users/tim/Desktop/continue.png',
            '/Users/tim/Desktop/find match1.png',
            #'/Users/tim/Desktop/sup.png',
            '/Users/tim/Desktop/yumi1.png',
            '/Users/tim/Desktop/Temo.png'
            '/Users/tim/Desktop/lock.png',
            '/Users/tim/Desktop/finish game.png',
            '/Users/tim/Desktop/next.png',
            '/Users/tim/Desktop/accept match.png',
            '/Users/tim/Desktop/ok.png',
        ]
        self.last_clicked_times = {}
        self.cooldown = 5
        self.sup_clicked = False
        self.yumi_clicked = False
        self.is_game_ready = False 

    def reset(self, seed=None, options=None):
        if seed is not None:
            np.random.seed(seed)
        self.prepare_game()
        initial_observation = self.get_state()  # get a inatioal observation value
        return initial_observation, {}  # return intial observaiton value and a info dictionary
      

    def prepare_game(self):
        if self.is_game_ready:
            print("Game is already ready, skipping setup.")
            return
        print("Starting automated game setup...")
        can_click_lock = False
        self.sup_clicked = False
        self.yumi_clicked = False
        while not can_click_lock:
            screenshot = np.array(pyautogui.screenshot())
            screen_gray = cv2.cvtColor(screenshot, cv2.COLOR_BGR2GRAY)
            for icon_path in self.icons_paths:
                if 'lock.png' in icon_path:
                    continue
                icon = cv2.imread(icon_path, cv2.IMREAD_GRAYSCALE)
                if icon is None:
                    print(f"Error loading image: {icon_path}")
                    continue
                res = cv2.matchTemplate(screen_gray, icon, cv2.TM_CCOEFF_NORMED)
                loc = np.where(res >= 0.8)
                for pt in zip(*loc[::-1]):
                    icon_name = icon_path.split('/')[-1]
                    logical_x = (pt[0] + icon.shape[1] // 2)//2
                    logical_y = (pt[1] + icon.shape[0] // 2)//2
                    current_time = time.time()
                    if icon_name in self.last_clicked_times and current_time - self.last_clicked_times[icon_name] < self.cooldown:
                        continue
                    pyautogui.moveTo(logical_x, logical_y, duration=0.5)
                    pyautogui.click(button='left')
                    self.last_clicked_times[icon_name] = current_time
                    if icon_path.endswith('Temo.png'):
                        self.sup_clicked = True
                    if icon_path.endswith('yumi1.png'):
                        self.yumi_clicked = True
        
            if self.sup_clicked or self.yumi_clicked:
                lock_icon = cv2.imread('/Users/tim/Desktop/lock.png', cv2.IMREAD_GRAYSCALE)
                lock_res = cv2.matchTemplate(screen_gray, lock_icon, cv2.TM_CCOEFF_NORMED)
                if np.max(lock_res) >= 0.8:
                    lock_loc = np.where(lock_res >= 0.8)
                    for pt in zip(*lock_loc[::-1]):
                        logical_x = (pt[0] + lock_icon.shape[1] // 2)//2
                        logical_y = (pt[1] + lock_icon.shape[0] // 2)//2
                        pyautogui.moveTo(logical_x, logical_y, duration=0.5)
                        pyautogui.click(button='left')
                        print("clicked lock, wait 2 mins for loading")
                        time.sleep(60)  # adjustable loading time
                        self.center_character()
                        self.is_game_ready = True
                        print("Game setup complete. Reinforcement learning starts...")
                        can_click_lock = True
                        return
 
                else:
                    print(f"Waiting for all conditions to be met: Sup Clicked: {self.sup_clicked}, Yumi Clicked: {self.yumi_clicked}")
  



    def center_character(self):
        print("Auto-clicking 'y' to center character...")
        self.keyboard.press('y')
        time.sleep(0.1)
        self.keyboard.release('y')
        time.sleep(1)

    def step(self, action):
        print(f"Action received: {action}")
        self.last_action = action  # remember last action
        self.execute_action(action)
        print("Action executed.")
        observation = self.get_state()  # get new state
        print("State updated.")
        reward = self.get_reward()  # caculate reward
        done = self.is_done()  # check if end
        truncated = False  #  if there is no special truncation condition, you can just set it to be the same as done or False.
        info = {}  # any additional statistical info
        print(f"Step complete: Observation shape: {observation.shape}, Reward: {reward}, Done: {done}")

        return observation, reward, done, truncated, info


    def execute_action(self, action):
        if action == 0:
            self.random_move_and_click() 
        elif action == 1:
            self.attack()  
        elif action == 2:
            self.keyboard.press('b')
            self.keyboard.release('b')  

    def get_state(self):
        with mss() as sct:
            monitor = {"top": 0, "left": 0, "width": 3024, "height": 1964}
            screenshot = sct.shot()
            frame = np.array(Image.open(screenshot))
            resized_frame = cv2.resize(frame, (3024, 1964))
            return cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)

    def get_reward(self):
        kills = self.extract_numbers(7, 2567, 19, 33)
        deaths = self.extract_numbers(7, 2597, 19, 33)
        minions = self.extract_numbers(10, 2766, 30, 30)
        health_current = self.extract_numbers(1865, 1300, 80, 40)
        health_max = self.extract_numbers(1865, 1400, 80, 40)
        health_percentage = health_current / health_max * 100 if health_max != 0 else 0

        reward = kills *10 - deaths*10 + minions * 5

 
        if health_percentage < 30 and self.last_action == 2:
            reward += 20
   
        elif health_percentage > 70 and self.last_action == 2:
            reward -= 20
        return reward 

    def is_done(self):
        if not self.is_game_ready:
            return False

        screen = self.get_state()
        screen_gray = cv2.cvtColor(screen, cv2.COLOR_BGR2GRAY)
        for end_game_icon in ['finish game.png', 'next.png']:
            template = cv2.imread(f'/Users/tim/Desktop/{end_game_icon}', cv2.IMREAD_GRAYSCALE)
            if template is None:
                continue
            result = cv2.matchTemplate(screen_gray, template, cv2.TM_CCOEFF_NORMED)
            _, max_val, _, max_loc = cv2.minMaxLoc(result)
            if np.max(result) >= 0.8:
                self.mouse.position = (max_loc[0], max_loc[1])
                self.mouse.click(Button.left)
                self.is_game_ready = False
                print("Game over detected. Preparing for the next game...")
                return True
        return False


    def random_move_and_click(self):
        x, y = np.random.randint(0, 3024), np.random.randint(0, 1964)
        self.mouse.position = (x//2, y//3)
        self.mouse.click(Button.right)

    def attack(self):
        x, y = np.random.randint(0, 3024), np.random.randint(0, 1964)
        self.mouse.position = (x//2, y//3)
        self.mouse.click(Button.right)

    def use_skill(self, key):
        self.keyboard.press(key)
        self.keyboard.release(key)

    def extract_numbers(self, top, left, width, height):
        with mss() as sct:
            monitor = {"top": top, "left": left, "width": width, "height": height}
            screenshot = sct.grab(monitor)
            image = np.array(screenshot)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            blur = cv2.GaussianBlur(gray, (5, 5), 0)
            _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            result_str = pytesseract.image_to_string(thresh, config='--oem 3 --psm 6 digits').strip()
            return int(result_str) if result_str.isdigit() else 0

In [7]:
import torch
print(torch.__version__)
print("Is MPS available:", torch.backends.mps.is_available())  # 对于 Apple Silicon


2.1.0.dev20230724
Is MPS available: True


In [2]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.evaluation import evaluate_policy

# creat enviorment and check env
env = MOBAEnv()
check_env(env)
env = DummyVecEnv([lambda: env])

# creat model
model = PPO("CnnPolicy", env, verbose=1, device='mps', batch_size=16, tensorboard_log="./MOBAEnv_tensorboard/") 
print("Start training...")
model.learn(total_timesteps=10000)
print("Training completed.")
model.save("ppo_MOBAEnv")

# load model
model = PPO.load("ppo_MOBAEnv")

# get initial observation value
observation = env.reset()[0]  # reset enviorment and get intial value
for i in range(10000):
    action, _states = model.predict(observation, bbdeterministic=True)
    observation, reward, done, info = env.step(action)[0]  # do action and get result
    print(f"Step: {i}, Reward: {reward}, Done: {done}")
    if done:
        observation = env.reset()[0]  # if the game is finished, reset the env


# evaluate model 
eval_env = DummyVecEnv([lambda: MOBAEnv()])
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward}, Std Reward: {std_reward}")


Starting automated game setup...
clicked lock, wait 2 mins for loading
Auto-clicking 'y' to center character...
Game setup complete. Reinforcement learning starts...
Game is already ready, skipping setup.
Action received: 1
Action executed.
State updated.
Step complete: Observation shape: (1964, 3024, 3), Reward: 0, Done: False
Game is already ready, skipping setup.
Action received: 0
Action executed.
State updated.
Step complete: Observation shape: (1964, 3024, 3), Reward: 0, Done: False
Action received: 1
Action executed.
State updated.
Step complete: Observation shape: (1964, 3024, 3), Reward: 0, Done: False
Action received: 2
Action executed.
State updated.
Step complete: Observation shape: (1964, 3024, 3), Reward: -20, Done: False
Action received: 2
Action executed.
State updated.
Step complete: Observation shape: (1964, 3024, 3), Reward: -20, Done: False
Action received: 2
Action executed.
State updated.
Step complete: Observation shape: (1964, 3024, 3), Reward: -20, Done: False


RuntimeError: Invalid buffer size: 11.05 GB